In [104]:
import pandas as pd
import numpy as np

#需要优化的量

#关系矩阵
#dts
#M1=np.zeros((19,106),dtype=int) #进口：对应1
#M3=np.zeros((19,106),dtype=int) #国产：对应2
#ctd
#M2=np.zeros((1,18),dtype=int) #进口
#M4=np.zeros((1,18),dtype=int) #国产

#仓库状态矩阵
#O=np.zeros((20,1),dtype=int)  #是否开仓
#Z=np.zeros((20,1),dtype=int)  #是否智能


In [105]:
#参数导入
#国产，对应2
Q_dm=pd.read_csv('data_p/Q_dm.csv')
Q_dm=Q_dm.to_numpy()
Q_dm=Q_dm[:,1]
Q_dm=Q_dm.reshape((1,106))
Q_dm.shape

(1, 106)

In [106]:
#港口，对应1
Q_im=pd.read_csv('data_p/Q_im.csv')
Q_im=Q_im.to_numpy()
Q_im=Q_im[:,1]
Q_im=Q_im.reshape((1,106))
Q_dm.shape

(1, 106)

In [107]:
L_dts=pd.read_csv('data_p/d_matrix_dts.csv')
L_dts=L_dts.to_numpy()
L_dts=L_dts[1:]
L_dts=L_dts[:,2:]
L_dts_1=np.delete(L_dts,11,axis=0)  #im(delete neilu)
#k = 10 
#row_to_move = L_dts_1[k]
#L_dts_1 = np.delete(L_dts_1, k, axis=0)
#L_dts_1 = np.vstack([row_to_move, L_dts_1])

L_dts_2=np.delete(L_dts,6,axis=0)  #dm(delete gangkou)
#k = 6 
#row_to_move = L_dts_2[k]
#L_dts_2 = np.delete(L_dts_2, k, axis=0)
#L_dts_2 = np.vstack([row_to_move, L_dts_2])
L_dts_2.shape

(19, 106)

In [108]:
T_dts=pd.read_csv('data_p/t_matrix_dts.csv')
T_dts=T_dts.to_numpy()
T_dts=T_dts[1:]
T_dts=T_dts[:,2:]
T_dts_1=np.delete(T_dts,11,axis=0)#im(delete neilu)
T_dts_2=np.delete(T_dts,6,axis=0)#dm(delete gangkou)
T_dts_1.shape

(19, 106)

In [109]:
L_ctd=pd.read_csv('data_p/d_matrix_ctd.csv')
L_ctd=L_ctd.to_numpy()
L_ctd=L_ctd[:,2:]

In [110]:
T_ctd=pd.read_csv('data_p/t_matrix_ctd.csv')
T_ctd=T_ctd.to_numpy()
T_ctd=T_ctd[:,2:]

In [111]:
Pf=pd.read_csv('data_p/Pf.csv')
Pf=Pf.to_numpy()
Pf=Pf[:,1]
Pf.shape

(20,)

In [112]:
Of=pd.read_csv('data_p/Of.csv')
Of=Of.to_numpy()
Of=Of[:,1]
#Of.reshape((1,20))
Of.shape

(20,)

In [113]:
Cap=pd.read_csv('data_p/Cap.csv')
Cap=Cap.to_numpy()
Cap=Cap[:,1]
#Cap.reshape((1,20))
Cap.shape

(20,)

In [114]:
#约束条件
##时间
Q_all=Q_im.sum()
def time_point(M_im,M_dm):
    Q_out=0
    tmp1=M_im*T_dts_1
    tmp2=M_dm*T_dts_2  #这里改到对应的时间矩阵
    #print(tmp1,"\n",tmp2)
    tmp1=tmp1.sum(axis=0)
    tmp2=tmp2.sum(axis=0)
    for i in range(0,106):
        if tmp1[i]>=10:
            Q_out+=Q_im[0,i]
        if tmp2[i]>=10:
            Q_out+=Q_dm[0,i]
    out_point=Q_out/Q_all
    return out_point

def time_lim(M_im,M_dm):
    if time_point(M_im,M_dm)<=0.05:
        return True
    return False

#最大订单量不超过最大承载量
def cap_lim(M_im,M_dm):
    tmp=M_im*Q_im
    tmp1=tmp.sum(axis=1)
    tmp2=M_dm*Q_dm
    tmp3=tmp2.sum(axis=1)
    """
    s1=0
    s2=0
    for j in range(0,20):
        if j==6:
            sum=tmp1[s1]
            s2-=1
        elif j==11:
            sum=tmp3[s2]
            s1-=1
        else:
            sum=tmp1[s1]+tmp3[s2]
        if sum>Cap[j]:
            return False
        s1+=1
        s2+=1
    """
    
    return True

#cdc必须开
def cdc_op(O):
    return O[6]==1 and O[11]==1

#每个门店对应一个商店
def sd_lim(M_1,M_3):
    M1_s=M_1.sum(axis=0)
    #print(M1_s.shape)
    M2_s=M_3.sum(axis=0)
    #print(M2_s)
    for i in M1_s:
        if i!=1:
            return False
    for i in M2_s:
        if i!=1:
            return False
    return True



In [115]:
#目标优化式
##处置成本
def ccl_cp(Z,Q,M_s,M_d,k,Of,Pf):
    tmp=M_s*Q  
    tmp1=tmp.sum(axis=1)
    #print(tmp1) #19,一维数组
    #tmp1=tmp1.reshape((1,19))
    tmp11=np.delete(tmp1,k)
    Pf=np.delete(Pf,k)
    Of=np.delete(Of,k)
    Z=np.delete(Z,k)
    #print(tmp11) 
    tmp2=M_d*tmp11
    ccp=tmp2.sum()
    """
    for i in range(0,19):
        fl=Z[i]
        if fl==0:
            cp+=fl+Of[i]+tmp1.T[i]*P[i][0]
        else:
            cp+=fl+Of[i]+tmp1.T[i]*P[i][1]
    """
    m=-0.5*Z+1
    cp=Of+tmp1*Pf*m+Z
    return cp+ccp
#运输成本
def ccl_ct(k,Q,M_s,M_d,L_s,L_d):
    #ct1=np.zeros((1,20))
    sum=0
    tmp1 = M_s * Q * L_s * 1.25
    Q_c = Q @ M_s.T
    Q_c = np.delete(Q_c,k,axis=1) 
    tmp2= M_d * Q_c * L_d.T * 0.6
    sum+=tmp1.sum()+tmp2.sum()
    return sum

#总成本
def cal_all(M1,M2,M3,M4,O,Z,Of,Pf,L_ctd,L_dts_1,L_dts_2):
    M1=M1.tolist()
    M1=np.array(M1)
    M2=M2.tolist()
    M2=np.array(M2)
    M3=M3.tolist()
    M3=np.array(M3)
    M4=M4.tolist()
    M4=np.array(M4)
    O=O.tolist()
    O=np.array(O)
    Z=Z.tolist()
    #print(Z[0])
    Z=np.array(Z)
    c=0
    #进口
    c+=ccl_cp(Z,Q_im,M1,M2,6,Of,Pf)
    #print(c)
    c+=ccl_ct(10,Q_im,M1,M2,L_dts_1,L_ctd[0])
    #print(c)
    #国产
    c+=ccl_cp(Z,Q_dm,M3,M4,10,Of,Pf)
    #print(c)
    c+=ccl_ct(6,Q_dm,M3,M4,L_dts_2,L_ctd[1])
    #print(c)
    return c[0]

In [116]:
import gurobipy as gp
from gurobipy import GRB

#2 个cdc
cdc_cnt=2
# 106 个store
s_cnt=106
#18个rdc
rdc_cnt=18
cdc_cnt_1=19
dc_cnt=20

# 创建模型
m = gp.Model("ass_mov")
# 创建变量
M1 = m.addMVar((cdc_cnt_1,s_cnt), vtype=GRB.BINARY,name="M1")
M2 = m.addMVar((cdc_cnt,rdc_cnt), vtype=GRB.BINARY,name="M2")
M3 = m.addMVar((cdc_cnt_1,s_cnt), vtype=GRB.BINARY,name="M3")
M4 = m.addMVar((cdc_cnt,rdc_cnt), vtype=GRB.BINARY,name="M4")
O = m.addMVar(dc_cnt, vtype=GRB.BINARY,name="O")
Z = m.addMVar(dc_cnt, vtype=GRB.BINARY,name="Z")
# 更新变量环境
m.update()
# 创建目标函数
m.setObjective(cal_all(M1,M2,M3,M4,O,Z,Of,Pf,L_ctd,L_dts_1,L_dts_2),GRB.MINIMIZE)
# 创建约束条件约束条件
m.addConstrs(time_lim(M1,M3)) 
m.addConstrs(cap_lim(M1,M3))
m.addConstrs(cdc_op(O))
m.addConstrs(sd_lim(M1,M3))
# 求解规划模型
m.optimize()

# 输出结果
solution = m.getAttr('M1',M1)



GurobiError: Constraint has no bool value (are you trying "lb <= expr <= ub"?)